In [7]:
from neo4j import GraphDatabase
IP="bolt://172.17.0.3:7687"
USER="neo4j"
PASSWORD="ahVUWKK8kKDhFUNz"
driver = GraphDatabase.driver(IP, auth=(USER, PASSWORD))

In [8]:
def clear_database(tx):
    tx.run("""MATCH (n) DETACH DELETE (n)""")

def add_tool(tx, name):
    tx.run("CREATE (node:Tool {name: $name}) RETURN node", name=name )

def print_tools(tx):
    res = ""
    for record in tx.run("""MATCH (t:Tool) RETURN t.name LIMIT 100"""):
        res = res + "; " + record["t.name"]
    print(res)

def lambda_request(tx, s):
    query_result = tx.run(s)
    return query_result

In [9]:
with driver.session() as session:
    session.write_transaction(clear_database)

In [10]:
with driver.session() as session:
    session.write_transaction(clear_database)
    session.write_transaction(add_tool, "tool1")
    session.write_transaction(print_tools)

; tool1


In [11]:
myRequest = """CREATE (t1:Tool {name:"t1"})-[:FUNCTION]->(f1:Function {name:"f1"})
                RETURN t1.name, f1.name"""

with driver.session() as session:
    query_result = session.write_transaction(lambda_request, myRequest)
    for returning in query_result:
        print(returning["t1.name"])
    session.write_transaction(print_tools)

; tool1; t1


In [12]:
driver.close()